### Lab 1. Detección de Pishing

#### Miembros
- Fernanda Esquivel - 21542
- Andrés Montoya - 21552

#### Link al repositorio
El repositorio puede ser visualizado [acá](https://github.com/FerEsq/SDS-Lab-01)

# Parte 1: Ingeniería de características

In [144]:
import pandas as pd
import math
from urllib.parse import urlparse
import re

## Exploración de datos

In [145]:
df = pd.read_csv('data/dataset_pishing.csv')

In [146]:
# 5 entries
df.head(5)

,url,status
0,http://www.crestonwood.com/router.php,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,phishing
2,https://support-appleld.com.secureupdate.duila...,phishing
3,http://rgipt.ac.in,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,legitimate


In [147]:
# count of legit and phishing
df['status'].value_counts()

status
legitimate    5715
phishing      5715
Name: count, dtype: int64

El dataset está balanceado. 

1. **¿Qué ventajas tiene el análisis de una URL contra el análisis de otros datos, cómo el tiempo de vida del dominio, o las características de la página Web?**
Analizar las url tiene muchas ventajas, entre ellas:
- Evita la descarga del sitio, el cual podría contener código malicioso.
- Es mucho más rapida al detectar en tiempo real.
- Es computacionalmente más eficaz ya que solo se deben procesar strings cortos.
2. **¿Qué características de una URL son más prometedoras para la detección de phishing?**
- Longitud de la url completa
- Longitud del dominio
- Cantidad de subdominios
- Cantidad o presencia de caracteres especiales
- Cantidad de números
- Cantidad de letras
- Protocolo
- Cantidad de directorios
- Presencia de parámetros
- Cantidad de párametros
- IP en la url
- TDL
- Identificación de redireccónes (posee más de una url)
- Presencia de credenciales en la url (correo)
- Entropía

## Derivación de características

In [148]:
# get the url length
df['url_length'] = df['url'].apply(lambda x: len(x))

In [149]:
# get the domain length
df['domain_length'] = df['url'].apply(lambda x: len(x.split('/')[2]))

In [150]:
# get protocol
df['protocol'] = df['url'].apply(lambda x: x.split(':')[0])

In [151]:
# get the number of subdomains
df['subdomains'] = df['url'].apply(lambda x: len(x.split('/')[2].split('.')) - 2)

In [152]:
# get if the url has a special characters
special_characters = ['@', '-', '_', '%', '?', '=', '&', '.']
df['special_characters'] = df['url'].apply(lambda x: 1 if any(c in x for c in special_characters) else 0)

In [153]:
# get the number of special characters
df['special_characters_count'] = df['url'].apply(lambda x: sum(1 for c in x if c in special_characters))

In [154]:
# get the number of digits
df['digits'] = df['url'].apply(lambda x: sum(c.isdigit() for c in x))

In [155]:
# get the number of letters
df['letters'] = df['url'].apply(lambda x: sum(c.isalpha() for c in x))

In [156]:
# get the number of directories, 
def get_directories(url: str):
    domain_and_directories = url.split('://', 1)[1].split('?')[0]
    directories = domain_and_directories.split('/')[1:]
    cleaned_directories = [d for d in directories if d != '']
    return cleaned_directories

In [157]:
def has_file_extension(url):
    dirs = get_directories(url)
    return any('.' in d for d in dirs)

In [158]:
# get if the url has a file extension
df['has_file_extension'] = df['url'].apply(lambda x: 1 if has_file_extension(x) else 0)

In [159]:
def count_file_extensions(url):
    dirs = get_directories(url)
    return sum(1 for dir in dirs if '.' in dir)

In [160]:
# get the number of file extensions
df['file_extension_count'] = df['url'].apply(count_file_extensions)

In [161]:
#cantidad de directorios
df['directories_count'] = df['url'].apply(lambda x: len(get_directories(x)))

In [162]:
#si hay y cantidad de parámetros
df['has_parameters'] = df['url'].apply(lambda x: 1 if '?' in x else 0)
df['parameters_count'] = df['url'].apply(lambda x: x.count('&') + 1 if '?' in x else 0)

In [163]:
#encontrar IP en la url
def has_ip(url):
    # Patrón para IPv4
    pattern = r'(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)'
    return 1 if re.search(pattern, url) else 0

In [164]:
df['has_ip'] = df['url'].apply(has_ip)

In [165]:
#encontrar TLD
def get_tld(url):
    domain = url.split('/')[2]
    return domain.split('.')[-1]

In [166]:
df['tld'] = df['url'].apply(get_tld)

In [167]:
#encontrar si hay redirecciones
df['has_multiple_urls'] = df['url'].apply(lambda x: 1 if x.count('//') > 1 else 0)

In [168]:
#encontrar si hay credenciales
def has_credentials(url):
    #buscar @ antes del dominio o patrones de email
    has_at = '@' in urlparse(url).netloc
    email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
    has_email = bool(re.search(email_pattern, url))
    return 1 if (has_at or has_email) else 0

In [169]:
df['has_credentials'] = df['url'].apply(has_credentials)

In [170]:
#obtener entropía
def calculate_entropy(url):
    #calcular frecuencia de cada carácter
    char_freq = {}
    for char in url:
        char_freq[char] = char_freq.get(char, 0) + 1
    
    #calcular entropía
    entropy = 0
    length = len(url)
    for freq in char_freq.values():
        probability = freq / length
        entropy -= probability * math.log2(probability)
    return entropy

In [171]:
df['entropy'] = df['url'].apply(calculate_entropy)

In [172]:
#mostrar primeras 5 lineas del dataset para visualizar los cambios
df.head(5)

,url,status,url_length,domain_length,protocol,subdomains,special_characters,special_characters_count,digits,letters,has_file_extension,file_extension_count,directories_count,has_parameters,parameters_count,has_ip,tld,has_multiple_urls,has_credentials,entropy
0,http://www.crestonwood.com/router.php,legitimate,37,19,http,1,1,3,0,30,1,1,1,0,0,0,com,0,0,3.787043
1,http://shadetreetechnology.com/V4/validation/a...,phishing,77,23,http,0,1,1,17,53,0,0,3,0,0,0,com,0,0,4.419864
2,https://support-appleld.com.secureupdate.duila...,phishing,126,50,https,3,1,13,19,88,0,0,2,1,3,0,com,0,0,4.753412
3,http://rgipt.ac.in,legitimate,18,11,http,1,1,2,0,13,0,0,0,0,0,0,in,0,0,3.461320
4,http://www.iracing.com/tracks/gateway-motorspo...,legitimate,55,15,http,1,1,4,0,45,0,0,2,0,0,0,com,0,0,4.097662


In [173]:
#guardar el dataset actualizado
df.to_csv('data/dataset_phishing_features.csv', index=False)

## Preprocesamiento

In [174]:
#legitimate = 0, phishing = 1
df['status'] = (df['status'] == 'phishing').astype(int)

In [175]:
#mostrar primeras 5 lineas del dataset para visualizar los cambios
df.head(5)

,url,status,url_length,domain_length,protocol,subdomains,special_characters,special_characters_count,digits,letters,has_file_extension,file_extension_count,directories_count,has_parameters,parameters_count,has_ip,tld,has_multiple_urls,has_credentials,entropy
0,http://www.crestonwood.com/router.php,0,37,19,http,1,1,3,0,30,1,1,1,0,0,0,com,0,0,3.787043
1,http://shadetreetechnology.com/V4/validation/a...,1,77,23,http,0,1,1,17,53,0,0,3,0,0,0,com,0,0,4.419864
2,https://support-appleld.com.secureupdate.duila...,1,126,50,https,3,1,13,19,88,0,0,2,1,3,0,com,0,0,4.753412
3,http://rgipt.ac.in,0,18,11,http,1,1,2,0,13,0,0,0,0,0,0,in,0,0,3.461320
4,http://www.iracing.com/tracks/gateway-motorspo...,0,55,15,http,1,1,4,0,45,0,0,2,0,0,0,com,0,0,4.097662


In [176]:
#guardar el dataset actualizado
df.to_csv('data/dataset_phishing_features.csv', index=False)

## Selección de Características

In [177]:
#eliminar duplicados (si existen)
df = df.drop_duplicates()

In [178]:
#elegir las características relevantes
selected_features = [
    'url',  #mantenemos la URL para referencia
    'status',  #variable objetivo
    'url_length',
    'domain_length',
    'protocol',
    'subdomains',
    'special_characters_count',
    'digits',
    'letters',
    'directories_count',
    'parameters_count',
    'has_ip',
    'tld',
    'has_multiple_urls',
    'has_credentials',
    'entropy'
]

In [179]:
#crear el nuevo dataset con las características seleccionadas
df_clean = df[selected_features]

In [180]:
#guardar el dataset limpio
df_clean.to_csv('data/dataset_phishing_features_clean.csv', index=False)

3. **¿Qué columnas o características fueron seleccionadas y por qué?**
* Medidas de longitud (url_length, domain_length): estas identifican URLs sospechosamente largas
* Características de dominio (protocol, subdomains, tld): estas analizan la estructura y seguridad del dominio
* Características de composición (special_characters_count, digits, letters): estas examinan los patrones de caracteres utilizados
* Características de estructura (directories_count, parameters_count): estas evalúan la complejidad de la URL
* Indicadores de seguridad (has_ip, has_multiple_urls, has_credentials): estas detectan prácticas sospechosas como el uso de IPs directas o redirecciones;
* Entroía: esta mide la aleatoriedad en la URL


Se eliminaron características redundantes como special_characters, has_parameters y has_file_extension, ya que sus versiones de conteo proporcionan información más detallada, resultando en un conjunto de características optimizado para identificar URLs maliciosas.

# Parte 2: Implementación

## Separación de datos